In [1]:
import cv2
import numpy as np
from keras.models import load_model

# Function to preprocess the image for model prediction
def preprocess_image(file_path):
    gray = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    if gray is None:
        raise ValueError(f"Image at path {file_path} could not be loaded.")
    
    resized = cv2.resize(gray, (224, 224))
    resized_rgb = np.stack((resized,)*3, axis=-1)
    resized_rgb = resized_rgb / 255.0
    return np.expand_dims(resized_rgb, axis=0)

# Function to make predictions using an ensemble of models
def ensemble_prediction(file_path, models):
    # Preprocess the image
    processed_image = preprocess_image(file_path)

    # Collect predictions from each model
    predictions = []
    for model in models:
        prediction = model.predict(processed_image)
        predictions.append(prediction)

    # Convert list of predictions to a numpy array for easier manipulation
    predictions = np.array(predictions)

    # Average the predictions across models
    average_prediction = np.mean(predictions, axis=0)

    # Get the predicted class index from the averaged predictions
    predicted_class_index = np.argmax(average_prediction, axis=1)
    return predicted_class_index

# Class mapping for predicted indices
class_mapping = ['healthy', 'leaf curl', 'leaf spot', 'whitefly', 'yellowish']

# Load your trained models
model_paths = ['../models/vgg/chilli.h5', 
               '../models/resnet/chilli.h5', 
               '../models/mixednet/chilli.h5',
               '../models/efficient/chilli.h5']

models = [load_model(model_path) for model_path in model_paths]

# Example file path for the image to predict
file_path = '../sample.jpg'

# Make a prediction using the ensemble
predicted_class_index = ensemble_prediction(file_path, models)

# Map the index to the corresponding class
predicted_class = class_mapping[predicted_class_index[0]]

# Print the predicted class
print(f'The predicted class is: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 852ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
The predicted class is: leaf spot
